In [1]:
from preprocess import *

# VerCors
VerCors is installed it in the shared folder in the VM. However, we made a symlink towards `/vercors`.

In [2]:
!/vercors/vct --help

[build.sc] [1/52] scriptSources 
[build.sc] [3/52] cliImports 
[build.sc] [15/52] compileResources 
[build.sc] [16/52] enclosingClasspath 
[build.sc] [20/52] resources.super.mill.scalalib.JavaModule.resources 
[build.sc] [23/52] mill.scalalib.ZincWorkerModule.zincLogDebug 
[build.sc] [24/52] mill.scalalib.ZincWorkerModule.worker 
[build.sc] [26/52] sources.super.mill.scalalib.JavaModule.sources 
[build.sc] [47/52] zincReportCachedProblems 
[3/654] vercors.packedResources 
[6/654] mill.scalalib.ZincWorkerModule.zincLogDebug 
[7/654] mill.scalalib.ZincWorkerModule.worker 
[11/654] hre.pprofProto.sources 
[13/654] mill.contrib.scalapblib.ScalaPBWorkerApi.scalaPBWorker 
[17/654] hre.sources 
[25/654] hre.pprofProto.scalaPBIncludePath 
[45/654] hre.pprofProto.compileResources 
[57/654] hre.pprofProto.zincReportCachedProblems 
[75/654] hre.compileResources 
[76/654] foreign-modules.project.fetchJars.z3.classPath 
[88/654] hre.zincReportCachedProblems 
[93/654] foreign-modules.project.colMeta

# Halide & Haliver
We build and installed in HaliVer/Halide in `\haliver`.
Let's see if we can write an Halide program and compile it

In [3]:
halide_program = """
#include "Halide.h"
#include <stdio.h>

using namespace Halide;
int main(int argc, char *argv[]) {

  ImageParam inp(type_of<int>(), 1, "inp"); 
  Func f("f");
  Var x("x");
  
  inp.requires(inp(_0) > 1);
  f(x) = inp(x)*inp(x);
  f.ensures(f(x) > 1);
  
  std::vector<Annotation> pipeline_anns = {
   requires(inp.dim(0).extent() == f.output_buffer().dim(0).extent()),
   requires(inp.dim(0).min() == f.output_buffer().dim(0).min())
  };
  
  f.translate_to_pvl("test_front.pvl", {inp}, {});
  f.compile_to_c("test_back.c", {inp}, pipeline_anns, "f");
} 
"""
with open("test/test.cpp", 'w') as f:
    f.write(halide_program)

In [4]:
# This should compile the program without errors
!g++ test/test.cpp -g -I /haliver/include -L  /haliver/lib -lHalide -lpthread -ldl -o test/test

In [5]:
# This should generate the source files
!cd test && LD_LIBRARY_PATH=/haliver/lib ./test

In [6]:
!ls test

test  test_back.c  test.cpp  test_front.log  test_front.pvl


In [7]:
# This should verify the front-end. It should say somewhere:
# `[INFO] Verification completed successfully.`
# Note that it is quite verbose
!/vercors/vct test/test_front.pvl

[build.sc] [1/52] scriptSources 
[build.sc] [3/52] cliImports 
[build.sc] [15/52] compileResources 
[build.sc] [16/52] enclosingClasspath 
[build.sc] [20/52] resources.super.mill.scalalib.JavaModule.resources 
[build.sc] [23/52] mill.scalalib.ZincWorkerModule.zincLogDebug 
[build.sc] [24/52] mill.scalalib.ZincWorkerModule.worker 
[build.sc] [26/52] sources.super.mill.scalalib.JavaModule.sources 
[build.sc] [47/52] zincReportCachedProblems 
[3/654] vercors.packedResources 
[6/654] mill.scalalib.ZincWorkerModule.zincLogDebug 
[7/654] mill.scalalib.ZincWorkerModule.worker 
[11/654] hre.pprofProto.sources 
[13/654] mill.contrib.scalapblib.ScalaPBWorkerApi.scalaPBWorker 
[17/654] hre.sources 
[25/654] hre.pprofProto.scalaPBIncludePath 
[45/654] hre.pprofProto.compileResources 
[57/654] hre.pprofProto.zincReportCachedProblems 
[75/654] hre.compileResources 
[76/654] foreign-modules.project.fetchJars.z3.classPath 
[88/654] hre.zincReportCachedProblems 
[93/654] foreign-modules.project.colMeta

[70.5%] [████████████████████████████████████████████████▏                   ]
VerCors › Transformation › simplify
[WARN] The binder `test_front.pvl:75:10`:`(\forall int x; 0 < f_min_0_124573125() + f_extent_0_553688150() ==> 1 < f_1972256518(x))` contains no triggers
[70.5%] [████████████████████████████████████████████████▏                   ]
VerCors › Transformation › simplify
[70.8%] [████████████████████████████████████████████████▍                   ]
VerCors › Transformation › simplify
[71.2%] [████████████████████████████████████████████████▋                   ]
VerCors › Transformation › classToRef
[71.3%] [████████████████████████████████████████████████▊                   ]
VerCors › Transformation › adtPointer
[71.4%] [████████████████████████████████████████████████▊                   ]
VerCors › Transformation › adtNothing
[71.4%] [████████████████████████████████████████████████▊                   ]
VerCors › Transformation › adtViper_order
[71.4%] [████████████████████

In [8]:
# Same for the back-end. It should say somewhere:
# `[INFO] Verification completed successfully.`
!/vercors/vct test/test_back.c

^C


In [9]:
# Since the output is so verbose, for the actual tests we use some scripting from preprocess.py
# The output gives back a tupple with (time, result). result is based of the error code of the command.
# For VerCors Verification Succes=0, Verification Fail=1, Error=2, TimeOut=3
# You can inspect the actual output in the log_file you provide.
runCommand(command=['/vercors/vct', 'test/test_front.pvl'], log_file='test/test_front.log', verbose=False)

KeyboardInterrupt: 

# Experiments from Paper

In [ ]:
# The experiments use the `Experiments` class, which run the produced files from cmake.
# The files are made via cmake, let's check if everything generates okay
# Warnings about clock skew you can ignore
!cmake -B build -S .
!cmake --build build

In [ ]:
# if this does not work, run 
#!rm -r build && mkdir build && cmake -B build -S . && cmake --build build

In [12]:
n = 1 # repetitions Normally set to 5, but 1 should be sufficient here
t = 1*60 # timeout in seconds
vercors_loc = '/vercors/vct'
silicon_loc = '/vercors/silicon'
# These are the versions
versions: Dict[str, List[str]] = {
    'blur' : ['0','1','2','3'],
    'hist' : ['0','1','2','3'],
    'conv_layer' : ['0','1','2','3'],
    'gemm' : ['0','1','2','3'],
    'auto_viz' : ['0','1','2','3'],
}
# these are the versions for memory only results
versionsMem: Dict[str, List[str]] = {
    'blur' : ['0','1','2','3'],
    'hist' : ['0','1','2','3'],
    'conv_layer' : ['0','1','2','3'],
    'gemm' : ['0','1','2','3'],
    'auto_viz' : ['0','1','2','3'],
    'camera_pipe' : [''],
    'bilateral_grid' : [''],
    'depthwise_separable_conv' : [''],
}
# Set up experiments class
experiments = Experiments(versions, versionsMem, vercors_loc=vercors_loc, 
                          silicon_loc=silicon_loc, repetitions = n, timeout = t)

In [ ]:
# You can run a front end example with
experiments.front_end(name='blur')
# A log of this run can be found in `logs/blur_front.pvl.txt`
# It is verifying the file `build/blur_front.pvl`

In [ ]:
# And a back-end example with
experiments.back_end(name='blur', version='0')
# A log of this run can be found in `logs/blur_0.c.txt`
# It is verifying the file `build/blur_0.c`

In [ ]:
# It takes the default values for timeout and repititions, but you can also set them directly.
# Furthermore, if you do not specify a version, it will run all versions.
# Note that the timeouts are actually set for the verification part of VerCors, for bigger files the 
# the tool takes 20 seconds to parse and translate before starting to verify.
experiments.back_end(name='blur', timeout=2, repetitions=1)

In [ ]:
# With the `mem` option set to True you run the mem versions
experiments.back_end(name='blur', version='1', mem=True, repetitions=2)
# A log of this run can be found in `logs/blur_0_mem.c.txt`

# Generate Table 1 & 2 by Loading

In [14]:
# The results can also be loaded. The results of the paper are located in
#'2023-10-09-18-59_results_verification.json' and
# `2023-10-09-18-59_line_info.json`
load_prefix = "results/2023-10-09-18-59"
experiments.load_results(load_prefix)

In [15]:
# It will warn about inconsistent results, as mentioned in the paper as well
mem_table = make_mem_table(experiments) # This is table 1

inconsistent results for 'gemm_2_mem.c'
avr_total, passes, fails,  timeouts, passtime, failtime: (196.43292951583862, 4, 1, 0, 196.250727891922, 197.16173601150513)


In [16]:
# This the actual latex table code
print(mem_table)
# We can write it to file
with open("result_table_mem.tex", "w") as f:
    f.write(mem_table)

\begin{tabular}{l l \vbar \vbar r \vbar r \vbar r r r r}
\hline \textbf{Name} & & \multicolumn{1}{l\vbar}{\textbf{\halide}} & \textbf{\textbf{Sched}}. & \multicolumn{3}{l}{\textbf{\c}} & \\
& & \textbf{LoC} & \textbf{Dir.} & \textbf{LoC} & \textbf{Ann.} & \textbf{Loops} & \textbf{T. (s).} \\ \hline \hline
blur & V0 & 36 & 0 & 178 &60 &2 & 18\\ \hline
 & V1-\{f,p\} & \ditto &2 &172 &56 &1 & 19\\ \hline
 & V2-\{c,p,r,s\} & \ditto &6 &212 &74 &6 & 29\\ \hline
 & V3-\{c,p,s,st,u\} & \ditto &8 &211 &72 &5 & 24\\ \hline
\hline
hist & V0 & 71 & 2 & 299 &98 &11 & 30\\ \hline
 & V1-\{c,p,r,u\} & \ditto &4 &308 &99 &11 & 38\\ \hline
 & V2-\{c,p,r,u\} & \ditto &6 &311 &105 &13 & 48\\ \hline
 & V3-\{c,p,r,u\} & \ditto &13 &312 &101 &13 & 48\\ \hline
\hline
conv\_ & V0 & 44 & 0 & 273 &148 &7 & 90\\ \hline
layer & V1-\{c,f,p,u\} & \ditto &4 &281 &145 &8 & 97\\ \hline
 & V2-\{p,r,s,u\} & \ditto &6 &302 &166 &10 & 209\\ \hline
 & V3-\{c,p,r,s,u\} & \ditto &15 &279 &148 &7 & 168\\ \hline
\hline
gemm & 

In [17]:
# Same for Table 2 of the paper
normal_table = make_normal_table(experiments)
# This the actual latex table code
print(normal_table)
# We can write it to file
with open("result_table.tex", "w") as f:
    f.write(normal_table)

inconsistent results for 'blur_3.c'
avr_total, passes, fails,  timeouts, passtime, failtime: (96.95583856105804, 4, 0, 1, 96.95583856105804, None)
inconsistent results for 'auto_viz_0.c'
avr_total, passes, fails,  timeouts, passtime, failtime: (151.5906184911728, 2, 0, 3, 151.5906184911728, None)
inconsistent results for 'auto_viz_2.c'
avr_total, passes, fails,  timeouts, passtime, failtime: (229.82511369387308, 3, 0, 2, 229.82511369387308, None)
inconsistent results for 'auto_viz_3.c'
avr_total, passes, fails,  timeouts, passtime, failtime: (192.47571301460266, 4, 0, 1, 192.47571301460266, None)
\begin{tabular}{l l \vbar \vbar r r \vbar r \vbar r \vbar r r r r \vbar \vbar r}
\hline Name & & \multicolumn{2}{l\vbar}{\halide} & \multicolumn{1}{l\vbar}{Fr-end} & Sched. & \multicolumn{3}{l}{\C} & & LoA \\
& & LoC & LoA & T. (s) & LoC & LoC & LoA & Loops & T. (s) & incr. \\ \hline \hline
blur & V0 & 36 & 2 & 8 & 0 & 178 &63 &2 & 21 & 31.5x\\ \hline
 & V1-\{f,p\} & \ditto & \ditto & \ditto &

In [18]:
# Make the pdf from the latex files
!pdflatex -quiet table.tex

pdflatex: unrecognized option '-quiet'
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./table.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/tools/xspace.sty)
(/usr/share/texlive/texmf-dist/tex/latex/l3backend/l3backend-pdftex.def)
No file table.aux.
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmtt.fd)
(./result_table_mem.tex)
Overfull \hbox (77.70122pt too wide) in paragraph at lines 1--21
 [][] 
(./result_table.tex)
Overfull \hbox (90.03825pt too wide) in paragraph at lines 1--29
 [][] 
[1{/var/lib/texmf/fonts/map/pdftex/updmap/pdftex.map}] [2] (./table.aux)

LaTe

mktexpk: Running mf-nowin -progname=mf \mode:=ljfour; mag:=1+0/600; nonstopmode; input ectt1000
This is METAFONT, Version 2.71828182 (TeX Live 2022/dev/Debian) (preloaded base=mf)

(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/ectt1000.mf
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exbase.mf)
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/ectt.mf
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exroman.mf
 Ok (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exaccess.mf
 Ok) (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/expseudo.mf
 Ok) (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exruwest.mf
 Ok [192] [193] [194] [195] [196] [197] [198] [199] [200] [201] [202] [203]
[204] [205] [206] [207] [208] [209] [210] [211] [212] [213] [214] [215]
[216] [217] [218] [219] [220] [221] [222] [223])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exrlwest.mf
 Ok [224] [225] [226] [227] [228] [229] [230] [231] [232] [233] [234] 

 Ok [14] [15] [19] [20] [13] [18] [33] [39] [42] [43] [44] [46] [47] [58]
[59] [61] [96] [189] [17] [45] [16] [21] [22])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exaccent.mf
 Ok [0] [1] [2] [3] [4] [5] [6] [7] [8] [9] [10] [11] [12])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exsign.mf
 Ok [24] [34] [35] [36] [37] [64] [191] [159])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exrlig.mf
 Ok [25] [26] [28] [27] [29] [30] [31])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exromp.mf
 Ok [38] [63] [190])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exrulett.mf
 Ok [65] [66] [67] [68] [69] [70] [71] [72] [73] [74] [75] [76] [77] [78]
[79] [80] [81] [82] [83] [84] [85] [86] [87] [88] [89] [90])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exrllett.mf
 Ok [97] [98] [99] [100] [101] [102] [103] [104] [105] [106] [107] [108]
[109] [110] [111] [112] [113] [114] [115] [116] [117] [118] [119] [120]
[121] [122])
(/usr/share/texl

In [19]:
# View the pdf here
PDF('table.pdf',size=(950,500))